---
# 서울시 내 도서관 프로그램 정보 크롤링
---

# 1차 크롤링

## 기본 정보 확인
- 대상 url : https://culture.seoul.go.kr/culture/culture/cultureEvent/list.do?searchCate=&menuNo=200110
    https://culture.seoul.go.kr/culture/culture/cultureEvent/list.do?pageIndex=1&menuNo=200011&isSearched=&searchDist=&searchCost=&searchField=EDUEXP&searchAge=&sdate=2022-01-01&edate=2024-09-09&searchStr=%EB%8F%84%EC%84%9C%EA%B4%80&field=EDUEXP
- robots.txt 결과
    User-agent: *
    
    Disallow: /
    
    Allow : /culture/culture/
    
    Allow : /culture/main/
    
    Allow : /culture/event/
    
    Allow : /culture/bbs/
    
    Allow : /culture/bosingak/
    
    Allow : /culture/festival/
    
    Allow : /resources/night/
    
    Allow : /night/main/
    
    Allow : /night/sub/
    
    Allow : /night/bbs/

## 라이브러리 호출

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import requests
import time
import pandas as pd

/var/folders/y6/tjfrq8vj411cyf6zll2ppq_w0000gn/T/ipykernel_2988/981951661.py:11: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## 칼럼 정리

In [2]:
lib_name = []        #도서관명
event_loc = []          #도서관명22
event_name = []       #프로그램 이름
event_target = []     #프로그램 참여 대상
event_cate = []       #프로그램 유형
event_date = []       #프로그램 일시
event_type = []
event_details = []

In [3]:
#이벤트 리스트 각각 붙들고있는거 -> uls id="dataList"
#이벤트 링크 -> li ahref

#링크 클릭 후 정보 섹션 1 -> div class "txt-box" -> "event_title"
# div class "event-title" 프로그램 타입(pclass "type") / 프로그램 명 (h2)

#정보 섹션 2 -> div class "type-box"
# 여기서 lis 따오기 -> 각각 div_class "type-th" / "type-td"으로 칼럼명 - 내용 수집 가능


## 1차 크롤링

In [103]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException

options = Options()
options.add_argument("disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36")

options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

driver = webdriver.Chrome(options=options)
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

url = "https://culture.seoul.go.kr/culture/culture/cultureEvent/list.do?pageIndex=1&menuNo=200011&isSearched=&searchDist=&searchCost=&searchField=EDUEXP&searchAge=&sdate=2022-01-01&edate=2024-07-31&searchStr=%EB%8F%84%EC%84%9C%EA%B4%80&field=EDUEXP"
driver.get(url)

element_present = EC.presence_of_element_located((By.CLASS_NAME, 'paginationSet'))
WebDriverWait(driver, 5).until(element_present)

try:
    events = []
    event_info = []
    
    for i in range(97):
        print(f'Page: {i+1} - Start')
        
        
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "dataList"))
        )
        
        # Re-locate the data list element to avoid stale references
        box = driver.find_element(By.ID, "dataList")
        event_links = box.find_elements(By.TAG_NAME, "li")
        
        for event_link in event_links:
            try:
                anchor = event_link.find_element(By.TAG_NAME, "a")
                href = anchor.get_attribute("href")
                print(href)
                events.append(href)
                info_text = anchor.find_element(By.CLASS_NAME, "txt")
                print(info_text.text)
                event_info.append(info_text.text)
            except StaleElementReferenceException:
                continue
        
        print(f'Page: {i+1} - Done')
         
        if i < 96:  # Stop before the last iteration
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#event-list > div.paginationSet > ul > li.i.next > a"))
            )
            next_btn.click()

            WebDriverWait(driver, 10).until(EC.staleness_of(next_btn))
        
finally:
    driver.quit()



Page: 1 - Start
Page: 1 - Done
Page: 2 - Start
https://culture.seoul.go.kr/culture/culture/cultureEvent/view.do?cultcode=148197&menuNo=200011&searchDist=&searchCost=&searchField=&searchAge=&sdate=2022-01-01&edate=2024-07-31&searchStr=%EB%8F%84%EC%84%9C%EA%B4%80&pageIndex=2
[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내
2024-07-27 ~ 2024-07-27
구립증산정보도서관
https://culture.seoul.go.kr/culture/culture/cultureEvent/view.do?cultcode=148251&menuNo=200011&searchDist=&searchCost=&searchField=&searchAge=&sdate=2022-01-01&edate=2024-07-31&searchStr=%EB%8F%84%EC%84%9C%EA%B4%80&pageIndex=2
[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕
2024-07-27 ~ 2024-07-27
서울도서관
https://culture.seoul.go.kr/culture/culture/cultureEvent/view.do?cultcode=148189&menuNo=200011&searchDist=&searchCost=&searchField=&searchAge=&sdate=2022-01-01&edate=2024-07-31&searchStr=%EB%8F%84%EC%84%9C%EA%B4%80&pageIndex=2
[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남
2024-07-26 ~ 2024-07-26
서울도서관
https://culture.seoul.go.kr/culture/culture/cultureEvent/view.do?

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001085c48b8 chromedriver + 5179576
1   chromedriver                        0x00000001085bc2ea chromedriver + 5145322
2   chromedriver                        0x00000001081332b0 chromedriver + 389808
3   chromedriver                        0x000000010817f3e1 chromedriver + 701409
4   chromedriver                        0x000000010817f691 chromedriver + 702097
5   chromedriver                        0x00000001081c1464 chromedriver + 971876
6   chromedriver                        0x00000001081a17dd chromedriver + 841693
7   chromedriver                        0x00000001081beb9b chromedriver + 961435
8   chromedriver                        0x00000001081a1553 chromedriver + 841043
9   chromedriver                        0x00000001081727f6 chromedriver + 649206
10  chromedriver                        0x000000010817305e chromedriver + 651358
11  chromedriver                        0x0000000108587b20 chromedr

In [104]:
print(len(events))
print(len(event_info))

946
946


In [106]:
event_info[0]

'[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~ 2024-07-27\n구립증산정보도서관'

In [132]:
import pandas as pd

events_df = pd.DataFrame(zip(events,event_info))
events_df.rename(columns = {0 : 'event_link',1:'event_info'}, inplace = True)
events_df

,event_link,event_info
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동..."
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...
...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...


In [110]:
events_df.to_csv('./output/events_raw.csv',index=False)

In [130]:
for info in event_info:
    lib_name = info.split("]")[0]
    event_title = info.split("]")[1]
    
    print(lib_name,"___",event_title)

# for split in (event_info[0].split("] ")):
#     print(split)
#     print("--")

[구립증산정보도서관 ___  7월 문화가 있는 영화관 영화상영안내
2024-07-27 ~ 2024-07-27
구립증산정보도서관
[서울도서관 ___  2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕
2024-07-27 ~ 2024-07-27
서울도서관
[서울도서관 ___  서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남
2024-07-26 ~ 2024-07-26
서울도서관
[마포구립서강도서관 ___ [7월/팟캐스트
[서초구립내곡도서관 ___  내곡도서관에서 여름을 즐기세요!
2024-07-26 ~ 2024-08-31
서초구립내곡도서관
[구산동도서관마을 ___  이달의 인문학: AI AGI 시대 알파세대 교육
2024-07-25 ~ 2024-07-25
구립구산동도서관마을
[서초구립내곡도서관 ___  SUMMER BREAK with Library
2024-07-25 ~ 2024-08-25
서초구립내곡도서관
[금천문화재단 ___  금천구립시흥도서관, 7월 문화가 있는 날 [시가 있는 예술가의 방
[강남구립논현도서관 ___  나를 돌보는 그림책 심리여행
2024-07-24 ~ 2024-10-02
강남구립논현도서관
[마포구립서강도서관 ___  [7-8월/어린이강좌
[제기동감초마을현진건기념도서관 ___  「도서관은 쿨하다 : 끄고, 도서관으로!」 캠페인 행사
2024-07-22 ~ 2024-08-31
제기동감초마을현진건기념도서관
[구립증산정보도서관 ___  2024년 제 7회 독서달리기 대회 [달려라 증산
[제기동감초마을현진건기념도서관 ___  BUDi Market
2024-07-22 ~ 2024-08-01
제기동감초마을현진건기념도서관
[구립증산정보도서관 ___  2024년 제 7회 독서달리기 대회 [달려라 증산
[구립증산정보도서관 ___  2024 참여예산 지원 사업 [그림책 보드게임 지도사 양성과정
[마포구립서강도서관 ___  [7-8월/행사
[국립어린이청소년도서관 ___  책읽어주세요!
2024-07-20 ~ 2024-07-21
서울아트책보고
[구립

IndexError: list index out of range

In [134]:
events_df['lib_name'] = events_df['event_info'].apply(lambda x: x.split("]")[0].replace("[",""))
events_df

,event_link,event_info,lib_name
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...,구립증산정보도서관
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...,서울도서관
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...,서울도서관
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동...",마포구립서강도서관
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...,서초구립내곡도서관
...,...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...,강남구립청담도서관
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...,강남구립청담도서관
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...,강남구립청담도서관


In [139]:
events_df.to_csv("./output/events_libnames.csv",index=False,encoding='cp949')

In [146]:
unique_libs = events_df['lib_name'].unique()
unique_libs = pd.DataFrame(unique_libs,columns=['lib_name'])
unique_libs

,lib_name
0,구립증산정보도서관
1,서울도서관
2,마포구립서강도서관
3,서초구립내곡도서관
4,구산동도서관마을
...,...
184,국립중앙도서관 실감서재 & 지식의 길
185,장안어린이도서관 2022년 봄학기 문화 프로그램 접수 안내\n2022-02-17 ~...
186,구립상림도서관 문화가 있는 날 비비디 바바디 북
187,강남구립청담도서관


In [149]:
unique_libs.to_csv('./output/unique_libs.csv',index=False,encoding='cp949')

In [147]:
import re
for lib in unique_libs['lib_name']:
    standard = re.complie("도서관")
    if lib 

구립증산정보도서관
서울도서관
마포구립서강도서관
서초구립내곡도서관
구산동도서관마을
금천문화재단
강남구립논현도서관
제기동감초마을현진건기념도서관
국립어린이청소년도서관
마포구립서강도서관 
휘경행복도서관
매봉산숲속도서관
장안어린이도서관
종로문화재단
성수도서관
성동구립도서관
구립구산동도서관마을
이문어린이도서관
구로기적의도서관
성동구립용답도서관
용두어울림작은도서관
2024년 성동구립용답도서관 지혜학교 한국문화의 기원을 찾아 떠나는 시간여행
2024 성동구 구립도서관 북 콘서트 썬킴 작가와 함께하는 '영화로 떠나는 중세 유럽 이야기
2024 성동구립용답도서관 길 위의 인문학 문화를 품은 세계 건축물
관악구통합도서관
용두어린이영어도서관
성동구립도서관·금호도서관 여름학기 문화강좌 운영
서울 문화의 밤
5월/팟캐스트
서울야외도서관 책냇가
성동문화재단
서초구립반포도서관
구로구립 궁동어린이도서관
마포구청
4월/팟캐스트
서울야외도서관
성동구 구립도서관
2024 도서관의 날·도서관주간
2024-04-11 ~ 2024-04-18
국립중앙도서관
궁동어린이도서관
도봉기적의도서관
논현도서관
김근태기념도서관
서울애니메이션센터
2월/팟캐스트
마포구립서강도서관 2월/일반강좌
2024 김근태기념도서관 인권의학연구소 연계전시 때아닌 모든 곳에서
마포구립서강도서관 2월/어린이메이킹
2024년 논현도서관 겨울 독서교실 겨울아, 읽자!
은평구립구산동도서관마을
마포구립서강도서관 1-2월
도봉문화재단
12월/서강도서관
12월 서강도서관 /어린이이벤트
12월 서강도서관 어린이메이킹
구로구립글마루한옥어린이도서관
도서관 시민관장과 함께하는 문화예술 프로그램&팝업도서관 나의 책을 찾아줘
마포구립서강도서관 11월/팟캐스트
2023 김근태기념도서관 민주화운동청년연합 창립 40주년 기억전 이제 다시, 일어나
마포구립서강도서관 11월
11월/Epi X 마포구립서강도서관
11월 마포구립서강도서관/어린이메이킹
마포구립서강도서관 11월 행사
서강도서관 10월/팟캐스트
마포구청 10월/도서관 초대석
마포구립서강도서관 10월
마포구립

---

# 2차 크롤링

## 이전 정보 호출

In [8]:
events_libnames = pd.read_csv('./output/events_libnames.csv',encoding='cp949')
unique_libs = pd.read_csv('./output/unique_libs.csv',encoding='cp949')

In [ ]:
events_libnames

In [15]:
# 대상 urls : 
events_libnames.loc[1,'event_link']

'https://culture.seoul.go.kr/culture/culture/cultureEvent/view.do?cultcode=148251&menuNo=200011&searchDist=&searchCost=&searchField=&searchAge=&sdate=2022-01-01&edate=2024-07-31&searchStr=%EB%8F%84%EC%84%9C%EA%B4%80&pageIndex=2'

### 칼럼 정리

In [29]:
#typebox : 'type-box'
events_place = [] #장소
events_period = [] #기간
events_time = [] #시간
events_target = []#대상
events_price = [] #요금
events_q = [] #문의

#상세 Txt : 'culture-content'
events_dummies = [] # 더미 정보

## 상세페이지당 정보 크롤링

In [15]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import requests
import time
import pandas as pd

In [16]:
#피쳐 정의 
events_box_info = {}
events_place = [] #장소
events_period = [] #기간
events_time = [] #시간
events_target = []#대상
events_price = [] #요금
events_q = [] #문의    

events_dummies = [] # 더미 정보

try:
    service = Service()
    options = webdriver.ChromeOptions()

    url = events_libnames.loc[767,'event_link']
    driver = webdriver.Chrome(service=service,options=options)
    driver.get(url)

    # 정보 크롤링
    # 페이지 로딩 기다리기
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'detail-btn'))
    WebDriverWait(driver, 5).until(element_present)
    print('page loaded')


    #섹션 1_typebox : 'type-box'
    type_box = driver.find_element(By.CLASS_NAME,"type-box")
    print(type_box)
    box_info = type_box.find_elements(By.TAG_NAME,"li")
    print(box_info)

    for info in box_info:
        table_h = info.find_element(By.CLASS_NAME,"type-th").text
        table_d = info.find_element(By.CLASS_NAME,"type-td").text
        events_box_info[table_h] = table_d
        print(events_box_info)

    for info_category in events_box_info.keys():
        # print(info_category)
        # print(events_box_info[info_category])
        
        if info_category == '장소': events_place.append(events_box_info[info_category])
        elif info_category == '기간': events_period.append(events_box_info[info_category])
        elif info_category == '시간': events_time.append(events_box_info[info_category])
        elif info_category == '대상': events_target.append(events_box_info[info_category])
        elif info_category == '요금': events_price.append(events_box_info[info_category])
        elif info_category == '문의': events_q.append(events_box_info[info_category])
        else : continue

    #섹션1_상세 txt : 'culture-content'
    dummy = driver.find_element(By.CLASS_NAME,'culture-content')
    print(dummy.text)
    events_dummies.append(dummy.text)

finally:
    driver.quit()
    

events_detail_df = pd.DataFrame(zip(url,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies))
events_detail_df.columns = ['event_link','events_place','events_period','events_time','events_target','events_price','events_q','events_dummies']
events_detail_df.head()

page loaded
<selenium.webdriver.remote.webelement.WebElement (session="4f342dce0c26b9892579b0b6d678a12e", element="f.924366C94C4A24BA9E74CA5E560F9078.d.7031A309784E225397BF3130C7E83E9E.e.28")>
[<selenium.webdriver.remote.webelement.WebElement (session="4f342dce0c26b9892579b0b6d678a12e", element="f.924366C94C4A24BA9E74CA5E560F9078.d.7031A309784E225397BF3130C7E83E9E.e.35")>, <selenium.webdriver.remote.webelement.WebElement (session="4f342dce0c26b9892579b0b6d678a12e", element="f.924366C94C4A24BA9E74CA5E560F9078.d.7031A309784E225397BF3130C7E83E9E.e.36")>, <selenium.webdriver.remote.webelement.WebElement (session="4f342dce0c26b9892579b0b6d678a12e", element="f.924366C94C4A24BA9E74CA5E560F9078.d.7031A309784E225397BF3130C7E83E9E.e.37")>, <selenium.webdriver.remote.webelement.WebElement (session="4f342dce0c26b9892579b0b6d678a12e", element="f.924366C94C4A24BA9E74CA5E560F9078.d.7031A309784E225397BF3130C7E83E9E.e.38")>, <selenium.webdriver.remote.webelement.WebElement (session="4f342dce0c26b989257

,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,h,3층 Book C,2022-07-07 ~ 2022-07-28,14:00 ~ 16:00,바느질 초보자인 성인 10명,무료,,"2022년 7월 Book Cx클래스\n\n한 땀 한 땀, 크래프티비즘\n\n지구를 ..."


In [22]:
print(events_box_info)

{'장소': '서울도서관 1층 생각마루', '기간': '2024-07-27 ~ 2024-07-27', '시간': '17:00-19:00', '대상': '누구나(어른+아이 1팀으로 구성)', '요금': '무료', '문의': '070-5413-5661'}


In [35]:
events_detail_df = pd.DataFrame(zip(url,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies))
events_detail_df.columns = ['event_link','events_place','events_period','events_time','events_target','events_price','events_q','events_dummies']
events_detail_df.head()




,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,h,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...


In [17]:
#피쳐 정의 
events_num = []

events_box_info = {}
events_place = [] #장소
events_period = [] #기간
events_time = [] #시간
events_target = []#대상
events_price = [] #요금
events_q = [] #문의    

events_dummies = [] # 더미 정보

for i in range(767,768):
# for i in range(767,len(events_libnames['event_link'])):
    try:
        events_num.append(i)
        print(i," : start")
        service = Service()
        options = webdriver.ChromeOptions()

        url = events_libnames.loc[i,'event_link']
        driver = webdriver.Chrome(service=service,options=options)
        driver.get(url)

        # 정보 크롤링
        # 페이지 로딩 기다리기
        time.sleep(0.05)
        # element_present = EC.presence_of_element_located((By.CLASS_NAME, 'detail-btn'))
        # WebDriverWait(driver, 10).until(element_present)
        print('page loaded')


        #섹션 1_typebox : 'type-box'
        type_box = driver.find_element(By.CLASS_NAME,"type-box")
        # print(type_box)
        box_info = type_box.find_elements(By.TAG_NAME,"li")
        # print(box_info)

        for info in box_info:
            table_h = info.find_element(By.CLASS_NAME,"type-th").text
            table_d = info.find_element(By.CLASS_NAME,"type-td").text
            events_box_info[table_h] = table_d
            # print(events_box_info)

        for info_category in events_box_info.keys():
            # print(info_category)
            # print(events_box_info[info_category])
            
            if info_category == '장소': events_place.append(events_box_info[info_category])
            elif info_category == '기간': events_period.append(events_box_info[info_category])
            elif info_category == '시간': events_time.append(events_box_info[info_category])
            elif info_category == '대상': events_target.append(events_box_info[info_category])
            elif info_category == '요금': events_price.append(events_box_info[info_category])
            elif info_category == '문의': events_q.append(events_box_info[info_category])
            else : continue

        #섹션1_상세 txt : 'culture-content'
        dummy = driver.find_element(By.CLASS_NAME,'culture-content')
        # print(dummy.text)
        events_dummies.append(dummy.text)
        
        print(i," : completed")
        
    finally:
        driver.quit()
    

events_detail_df = pd.DataFrame(zip(events_num,events_libnames['event_link'][768:],events_place,events_period,events_time,events_target,events_price,events_q,events_dummies))
events_detail_df.columns = ['events_num','event_link','events_place','events_period','events_time','events_target','events_price','events_q','events_dummies']
events_detail_df.head()

767  : start
page loaded
767  : completed


,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,767,https://culture.seoul.go.kr/culture/culture/cu...,3층 Book C,2022-07-07 ~ 2022-07-28,14:00 ~ 16:00,바느질 초보자인 성인 10명,무료,,"2022년 7월 Book Cx클래스\n\n한 땀 한 땀, 크래프티비즘\n\n지구를 ..."


In [20]:
events_out = events_detail_df.copy()
events_out.to_csv('./output/temp_767.csv',index=False,encoding='utf-8-sig')

In [61]:
events_detail_df = pd.DataFrame(zip(events_num,events_libnames['event_link'][630:767],events_place,events_period,events_time,events_target,events_price,events_q,events_dummies))
events_detail_df.columns = ['events_num','event_link','events_place','events_period','events_time','events_target','events_price','events_q','events_dummies']
events_detail_df.head()

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,630,https://culture.seoul.go.kr/culture/culture/cu...,지하 문화강좌실,2022-12-14 ~ 2022-12-14,오후 7시 ~ 8시 30분,초등 1~3학년 어린이 및 보호자 12쌍,무료,,칼림바 동아리 손끝에서 와 함께 하는 <칼림바 체험의 밤>\n\n나 혼자 칼림바 친...
1,631,https://culture.seoul.go.kr/culture/culture/cu...,서울 동대문구 장안벚꽃로 323 휘경행복도서관,2022-12-14 ~ 2022-12-14,10:00~12:00,시니어(60세 이상 성인) / 10명,무료,,나의 작은 정원 - 시니어 특강\n\n크리스마스\n테라리움\n만들기\n\n60세 이...
2,632,https://culture.seoul.go.kr/culture/culture/cu...,글빛정보도서관 1층 별다방,2022-12-14 ~ 2022-12-14,저녁 7시 30분 ~ 저녁 9시,누구나,무료,,"""나는 어떤 모양을 가지고 있나요?""\n\n타인이 모르는 나의 내면을 그림으로 나타..."
3,633,https://culture.seoul.go.kr/culture/culture/cu...,지하1층 문화강좌실,2022-12-12 ~ 2022-12-13,오후 7시 ~ 8시 30분,초등학생 5~6학년 12명,무료,,<어린이 팝아트 초상화 그리기 체험>\n\n팝아트에 대하여 알아보고 초상화를 그려보...
4,634,https://culture.seoul.go.kr/culture/culture/cu...,지하 문화강좌실,2022-12-10 ~ 2022-12-10,오전 11시 ~ 12시,7~9세 유아 및 어린이 12명,무료,,"마음 이끄미 선생님과 함께 하는 그림책감정표현놀이프로그램\n마음아, 놀자!\n\n그..."


In [65]:
events_third = events_detail_df.copy()
events_third

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,768,https://culture.seoul.go.kr/culture/culture/cu...,마포구립서강도서관 등,2022-07-06 ~ 2022-09-17,09:00-18:00,일반 30명,무료,,"다시, 여기, 지금 [K-마포]\n\n\n강유정, 심두보, 오승현, 한기호, 송현민..."
1,769,https://culture.seoul.go.kr/culture/culture/cu...,꿈마을도서관 3층 프로그램실,2022-07-06 ~ 2022-07-27,10:30 - 12:00,성인,무료,,[구로구립]꿈마을도서관 <어른도 그림책 #여름>\n일시 : 22.07.06(수) -...
2,770,https://culture.seoul.go.kr/culture/culture/cu...,3층 세미나실,2022-07-06 ~ 2022-07-06,19:00 ~ 21:00,일반 30명,무료,,"다시, 여기, 지금 [K-마포]\n\nK-Culture, 빛과 그림자\n\n \n ..."
3,771,https://culture.seoul.go.kr/culture/culture/cu...,꿈마을도서관 3층 프로그램실,2022-07-05 ~ 2022-07-26,10:30 - 12:00,성인,무료,,[구로구립]꿈마을도서관 <미술심리로 보는 나 #7월>\n일시 : 22.07.05(화...
4,772,https://culture.seoul.go.kr/culture/culture/cu...,우리소리도서관,2022-07-02 ~ 2022-07-30,09:30,초등학교 3-4학년,무료,02-6263-1184,"꿈다락 토요문화학교 <할미가 간다, 덩 더 쿵따라기>는 우리 아이들이 또래 친구들과..."
...,...,...,...,...,...,...,...,...,...
173,941,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,
174,942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태..."
175,943,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,
176,944,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,


In [66]:
events_third.to_csv('./output/temp_third.csv',index=False,encoding='utf-8-sig')

---
# 데이터 취합
---

In [43]:
import pandas as pd
events_first = pd.read_csv('./output/temp.csv')
events_second = pd.read_csv('./output/temp_second.csv')
events_third = pd.read_csv('./output/temp_third.csv')
events_libnames = pd.read_csv('./output/events_libnames.csv',encoding='cp949')
events_out = pd.read_csv('./output/temp_767.csv')
unique_libs = pd.read_csv('./output/unique_libs.csv',encoding='cp949')

In [44]:
events_first

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,0,https://culture.seoul.go.kr/culture/culture/cu...,2층 소강당,2024-07-27 ~ 2024-07-27,14:00~16:00,도서관 이용자,무료,02-307-6030,7월 문화가 있는 영화관 영화상영안내\n 7월27일 토요일 오후 2시 영화...
1,1,https://culture.seoul.go.kr/culture/culture/cu...,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...
2,2,https://culture.seoul.go.kr/culture/culture/cu...,서울도서관 1층 생각마루,2024-07-26 ~ 2024-07-26,19:00 ~ 20:30,서울도서관 회원,무료,02-2133-0248,서울도서관 7월 마지막주 「서울 문화의 밤」 행사로 《서울의 자서전》 신병주 작가와...
3,3,https://culture.seoul.go.kr/culture/culture/cu...,온라인,2024-07-26 ~ 2024-08-20,09:00 ~ 18:00,누구나,무료,02-3141-7053,"서강도서관피셜 마흔 일곱 번째 시간\n정경아 작가의 <일주일에 세 번, 동네문화..."
4,4,https://culture.seoul.go.kr/culture/culture/cu...,서초구립내곡도서관,2024-07-26 ~ 2024-08-31,프로그램 별 상이 / 내곡도서관 매 주 월요일 휴관,유아·어린이,"원데이베이킹 재료비 15,000원",02-3461-3072,SUMMER BREAK WITH LIBRARY & 도서관에서 여름나기 \n ...
...,...,...,...,...,...,...,...,...,...
625,625,https://culture.seoul.go.kr/culture/culture/cu...,어린이열람실/ 모자열람실,2022-12-27 ~ 2022-12-27,오후 3시 ~ 재료 소진 시,유아 및 어린이 이용자 누구나 (선착순),무료,NaN,연말 기념 도서관 팝업 행사\n반짝!하고 열리는 반짝반짝 키링 만들기\n\n도서관 ...
626,626,https://culture.seoul.go.kr/culture/culture/cu...,3층 어린이열람실,2022-12-19 ~ 2022-12-24,개관시간 내,도서관 어린이 이용자 누구나,무료,NaN,크리스마스 맞이 <함께 만드는 크리스마스>\n\n호일아트를 완성하여 크리스마스 트리...
627,627,https://culture.seoul.go.kr/culture/culture/cu...,"김근태기념도서관 2층 공간,마루(강당)",2022-12-17 ~ 2022-12-17,13:00~15:00,"어린이, 청소년, 성인",무료,070-7718-8147,김근태기념도서관 개관 1주년 기념전 및\n민주주의자 故김근태 선생 11주기 추모전《...
628,628,https://culture.seoul.go.kr/culture/culture/cu...,도서관 1층 로비,2022-12-17 ~ 2022-12-17,오후 3시 ~ 재료 소진 시,도서관 어린이 이용자 누구나,무료,NaN,크리스마스 맞이 로비 체험 행사\n<소복소복 눈 내린 성탄 거울 만들기>\n\n어느...


In [45]:
events_second

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,630,https://culture.seoul.go.kr/culture/culture/cu...,지하 문화강좌실,2022-12-14 ~ 2022-12-14,오후 7시 ~ 8시 30분,초등 1~3학년 어린이 및 보호자 12쌍,무료,NaN,칼림바 동아리 손끝에서 와 함께 하는 <칼림바 체험의 밤>\n\n나 혼자 칼림바 친...
1,631,https://culture.seoul.go.kr/culture/culture/cu...,서울 동대문구 장안벚꽃로 323 휘경행복도서관,2022-12-14 ~ 2022-12-14,10:00~12:00,시니어(60세 이상 성인) / 10명,무료,NaN,나의 작은 정원 - 시니어 특강\n\n크리스마스\n테라리움\n만들기\n\n60세 이...
2,632,https://culture.seoul.go.kr/culture/culture/cu...,글빛정보도서관 1층 별다방,2022-12-14 ~ 2022-12-14,저녁 7시 30분 ~ 저녁 9시,누구나,무료,NaN,"""나는 어떤 모양을 가지고 있나요?""\n\n타인이 모르는 나의 내면을 그림으로 나타..."
3,633,https://culture.seoul.go.kr/culture/culture/cu...,지하1층 문화강좌실,2022-12-12 ~ 2022-12-13,오후 7시 ~ 8시 30분,초등학생 5~6학년 12명,무료,NaN,<어린이 팝아트 초상화 그리기 체험>\n\n팝아트에 대하여 알아보고 초상화를 그려보...
4,634,https://culture.seoul.go.kr/culture/culture/cu...,지하 문화강좌실,2022-12-10 ~ 2022-12-10,오전 11시 ~ 12시,7~9세 유아 및 어린이 12명,무료,NaN,"마음 이끄미 선생님과 함께 하는 그림책감정표현놀이프로그램\n마음아, 놀자!\n\n그..."
...,...,...,...,...,...,...,...,...,...
132,762,https://culture.seoul.go.kr/culture/culture/cu...,지하문화강좌실,2022-07-16 ~ 2022-07-16,오전 10~12시,초등 3 ~ 5학년 어린이 15명,무료,NaN,"제5회 독서달리기대회 <달려라, 증산> 개최 기념\n\n독후감 15줄 쓰기의 비법\..."
133,763,https://culture.seoul.go.kr/culture/culture/cu...,지하 1층 문화강좌실 및 야외,2022-07-16 ~ 2022-08-06,오후 1시 30분 ~ 2시 30분 매주 토요일(4회차),관심있는 초등학교 고학년 및 청소년 누구나 10명(2004년생~2012년생),무료,NaN,명지대학교 재학생들과 함께하는 M-J지식나눔 프로그램이 시작됩니다^_^\n다양한 콘...
134,764,https://culture.seoul.go.kr/culture/culture/cu...,김근태기념도서관 로비,2022-07-15 ~ 2022-08-13,"매주 금, 토, 일요일 11시 / 14시 / 16시",청소년 및 성인,무료,070-7718-8147,"김근태기념도서관 라키비움 해설 프로그램 <도슨트 ON> 참여자 모집\n\n"" 라키비..."
135,765,https://culture.seoul.go.kr/culture/culture/cu...,3층 세미나실,2022-07-13 ~ 2022-06-13,19:00 ~ 21:00,일반 30명,무료,NaN,"다시, 여기, 지금 [K-마포]\n\nK-Wave, 타인의 시선\n\n \n \n현..."


In [46]:
events_third

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,768,https://culture.seoul.go.kr/culture/culture/cu...,마포구립서강도서관 등,2022-07-06 ~ 2022-09-17,09:00-18:00,일반 30명,무료,NaN,"다시, 여기, 지금 [K-마포]\n\n\n강유정, 심두보, 오승현, 한기호, 송현민..."
1,769,https://culture.seoul.go.kr/culture/culture/cu...,꿈마을도서관 3층 프로그램실,2022-07-06 ~ 2022-07-27,10:30 - 12:00,성인,무료,NaN,[구로구립]꿈마을도서관 <어른도 그림책 #여름>\n일시 : 22.07.06(수) -...
2,770,https://culture.seoul.go.kr/culture/culture/cu...,3층 세미나실,2022-07-06 ~ 2022-07-06,19:00 ~ 21:00,일반 30명,무료,NaN,"다시, 여기, 지금 [K-마포]\n\nK-Culture, 빛과 그림자\n\n \n ..."
3,771,https://culture.seoul.go.kr/culture/culture/cu...,꿈마을도서관 3층 프로그램실,2022-07-05 ~ 2022-07-26,10:30 - 12:00,성인,무료,NaN,[구로구립]꿈마을도서관 <미술심리로 보는 나 #7월>\n일시 : 22.07.05(화...
4,772,https://culture.seoul.go.kr/culture/culture/cu...,우리소리도서관,2022-07-02 ~ 2022-07-30,09:30,초등학교 3-4학년,무료,02-6263-1184,"꿈다락 토요문화학교 <할미가 간다, 덩 더 쿵따라기>는 우리 아이들이 또래 친구들과..."
...,...,...,...,...,...,...,...,...,...
173,941,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,NaN
174,942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,NaN,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태..."
175,943,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,NaN
176,944,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,NaN


In [47]:
events_out

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,767,https://culture.seoul.go.kr/culture/culture/cu...,3층 Book C,2022-07-07 ~ 2022-07-28,14:00 ~ 16:00,바느질 초보자인 성인 10명,무료,NaN,"2022년 7월 Book Cx클래스\n\n한 땀 한 땀, 크래프티비즘\n\n지구를 ..."


In [48]:
print(len(events_first) + len(events_second)+len(events_third))
print(len(events_libnames))

945
946


In [49]:
print(len(events_first) + len(events_second)+len(events_third)+len(events_out))
print(len(events_libnames))

946
946


In [50]:
events_order = pd.concat([events_first,events_second],axis=0)
events_order = pd.concat([events_order,events_out],axis=0)
events_order = pd.concat([events_order,events_third],axis=0)
events_order

,events_num,event_link,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,0,https://culture.seoul.go.kr/culture/culture/cu...,2층 소강당,2024-07-27 ~ 2024-07-27,14:00~16:00,도서관 이용자,무료,02-307-6030,7월 문화가 있는 영화관 영화상영안내\n 7월27일 토요일 오후 2시 영화...
1,1,https://culture.seoul.go.kr/culture/culture/cu...,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...
2,2,https://culture.seoul.go.kr/culture/culture/cu...,서울도서관 1층 생각마루,2024-07-26 ~ 2024-07-26,19:00 ~ 20:30,서울도서관 회원,무료,02-2133-0248,서울도서관 7월 마지막주 「서울 문화의 밤」 행사로 《서울의 자서전》 신병주 작가와...
3,3,https://culture.seoul.go.kr/culture/culture/cu...,온라인,2024-07-26 ~ 2024-08-20,09:00 ~ 18:00,누구나,무료,02-3141-7053,"서강도서관피셜 마흔 일곱 번째 시간\n정경아 작가의 <일주일에 세 번, 동네문화..."
4,4,https://culture.seoul.go.kr/culture/culture/cu...,서초구립내곡도서관,2024-07-26 ~ 2024-08-31,프로그램 별 상이 / 내곡도서관 매 주 월요일 휴관,유아·어린이,"원데이베이킹 재료비 15,000원",02-3461-3072,SUMMER BREAK WITH LIBRARY & 도서관에서 여름나기 \n ...
...,...,...,...,...,...,...,...,...,...
173,941,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,NaN
174,942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,NaN,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태..."
175,943,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,NaN
176,944,https://culture.seoul.go.kr/culture/culture/cu...,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,NaN


In [51]:
print(len(events_order))

946


In [52]:
events_libnames

,event_link,event_info,lib_name
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...,구립증산정보도서관
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...,서울도서관
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...,서울도서관
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동...",마포구립서강도서관
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...,서초구립내곡도서관
...,...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...,강남구립청담도서관
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...,강남구립청담도서관
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...,강남구립청담도서관


In [53]:
events_whole = events_libnames.merge(events_order,on='event_link')
events_whole

,event_link,event_info,lib_name,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...,구립증산정보도서관,0,2층 소강당,2024-07-27 ~ 2024-07-27,14:00~16:00,도서관 이용자,무료,02-307-6030,7월 문화가 있는 영화관 영화상영안내\n 7월27일 토요일 오후 2시 영화...
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...,서울도서관,1,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...,서울도서관,2,서울도서관 1층 생각마루,2024-07-26 ~ 2024-07-26,19:00 ~ 20:30,서울도서관 회원,무료,02-2133-0248,서울도서관 7월 마지막주 「서울 문화의 밤」 행사로 《서울의 자서전》 신병주 작가와...
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동...",마포구립서강도서관,3,온라인,2024-07-26 ~ 2024-08-20,09:00 ~ 18:00,누구나,무료,02-3141-7053,"서강도서관피셜 마흔 일곱 번째 시간\n정경아 작가의 <일주일에 세 번, 동네문화..."
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...,서초구립내곡도서관,4,서초구립내곡도서관,2024-07-26 ~ 2024-08-31,프로그램 별 상이 / 내곡도서관 매 주 월요일 휴관,유아·어린이,"원데이베이킹 재료비 15,000원",02-3461-3072,SUMMER BREAK WITH LIBRARY & 도서관에서 여름나기 \n ...
...,...,...,...,...,...,...,...,...,...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...,강남구립청담도서관,941,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,NaN
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,942,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,NaN,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태..."
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...,강남구립청담도서관,943,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,NaN
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...,강남구립청담도서관,944,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,NaN


In [54]:
events_whole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946 entries, 0 to 945
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   event_link      946 non-null    object
 1   event_info      946 non-null    object
 2   lib_name        946 non-null    object
 3   events_num      946 non-null    int64 
 4   events_place    946 non-null    object
 5   events_period   946 non-null    object
 6   events_time     946 non-null    object
 7   events_target   946 non-null    object
 8   events_price    946 non-null    object
 9   events_q        579 non-null    object
 10  events_dummies  939 non-null    object
dtypes: int64(1), object(10)
memory usage: 81.4+ KB


In [67]:
events_whole.to_csv('./output/event_info.csv',index=False,encoding='utf-8-sig')

---
# 추가 전처리
---

In [91]:
events_whole = pd.read_csv('./output/event_info.csv')
events_whole

,event_link,event_info,lib_name,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...,구립증산정보도서관,0,2층 소강당,2024-07-27 ~ 2024-07-27,14:00~16:00,도서관 이용자,무료,02-307-6030,7월 문화가 있는 영화관 영화상영안내\n 7월27일 토요일 오후 2시 영화...
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...,서울도서관,1,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...,서울도서관,2,서울도서관 1층 생각마루,2024-07-26 ~ 2024-07-26,19:00 ~ 20:30,서울도서관 회원,무료,02-2133-0248,서울도서관 7월 마지막주 「서울 문화의 밤」 행사로 《서울의 자서전》 신병주 작가와...
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동...",마포구립서강도서관,3,온라인,2024-07-26 ~ 2024-08-20,09:00 ~ 18:00,누구나,무료,02-3141-7053,"서강도서관피셜 마흔 일곱 번째 시간\n정경아 작가의 <일주일에 세 번, 동네문화..."
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...,서초구립내곡도서관,4,서초구립내곡도서관,2024-07-26 ~ 2024-08-31,프로그램 별 상이 / 내곡도서관 매 주 월요일 휴관,유아·어린이,"원데이베이킹 재료비 15,000원",02-3461-3072,SUMMER BREAK WITH LIBRARY & 도서관에서 여름나기 \n ...
...,...,...,...,...,...,...,...,...,...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...,강남구립청담도서관,941,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,NaN
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,942,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,NaN,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태..."
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...,강남구립청담도서관,943,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,NaN
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...,강남구립청담도서관,944,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,NaN


## 시작일 - 종료일 분리

In [92]:
events_whole['start_date'] = events_whole['events_period'].apply(lambda x: x.split("~")[0]).replace(" ","")
events_whole['end_date'] = events_whole['events_period'].apply(lambda x: x.split("~")[1]).replace(" ","")
events_whole

,event_link,event_info,lib_name,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies,start_date,end_date
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...,구립증산정보도서관,0,2층 소강당,2024-07-27 ~ 2024-07-27,14:00~16:00,도서관 이용자,무료,02-307-6030,7월 문화가 있는 영화관 영화상영안내\n 7월27일 토요일 오후 2시 영화...,2024-07-27,2024-07-27
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...,서울도서관,1,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...,2024-07-27,2024-07-27
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...,서울도서관,2,서울도서관 1층 생각마루,2024-07-26 ~ 2024-07-26,19:00 ~ 20:30,서울도서관 회원,무료,02-2133-0248,서울도서관 7월 마지막주 「서울 문화의 밤」 행사로 《서울의 자서전》 신병주 작가와...,2024-07-26,2024-07-26
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동...",마포구립서강도서관,3,온라인,2024-07-26 ~ 2024-08-20,09:00 ~ 18:00,누구나,무료,02-3141-7053,"서강도서관피셜 마흔 일곱 번째 시간\n정경아 작가의 <일주일에 세 번, 동네문화...",2024-07-26,2024-08-20
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...,서초구립내곡도서관,4,서초구립내곡도서관,2024-07-26 ~ 2024-08-31,프로그램 별 상이 / 내곡도서관 매 주 월요일 휴관,유아·어린이,"원데이베이킹 재료비 15,000원",02-3461-3072,SUMMER BREAK WITH LIBRARY & 도서관에서 여름나기 \n ...,2024-07-26,2024-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...,강남구립청담도서관,941,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,NaN,2021-03-25,2022-02-24
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,942,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,NaN,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태...",2021-03-23,2022-12-31
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...,강남구립청담도서관,943,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,NaN,2021-03-14,2022-02-13
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...,강남구립청담도서관,944,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,NaN,2021-03-14,2022-02-13


In [93]:
events_whole.to_csv('./output/event_info.csv',index=False,encoding='utf-8-sig')

## 범주형 데이터 정리

In [75]:
events_whole['events_price'].value_counts()

events_price
무료                                           910
만화의집(무료) / 유료(애니소풍) 성인 4,000원, 어린이 6,000원     16
없음                                             5
강좌별 상이                                         2
원데이베이킹 재료비 15,000원                             1
재료비 10,000원                                    1
비누만들기 프로그램 10,000원 외 무료                        1
원데이베이킹 15,000원 / 나머지 프로그램 무료                   1
홈페이지 참고                                        1
64,000                                         1
재료비 3,000원                                     1
전석 25,000원                                     1
재료비 20.000원                                    1
40000원~64000원                                  1
전석 10,000원                                     1
무료 (동대문구립도서관 회원카드 발급자에 한해 신청가능)                1
교재비(20,000원), 자격검정료(80,000원) 별도                1
Name: count, dtype: int64

In [90]:
events_whole['events_target'].value_counts()

events_target
누구나                      156
성인                        50
어린이                       42
도서관 이용자                   31
유아                        25
                        ... 
누구나 (30명)                  1
초등 3~6학년/ 12명              1
유아 및 아동(각 수업 별 인원 상이)      1
성인 (20명)                   1
6세~13세, 15명                1
Name: count, Length: 410, dtype: int64

In [78]:
print(events_whole['events_target'].unique())

['도서관 이용자' '누구나(어른+아이 1팀으로 구성)' '서울도서관 회원' '누구나' '유아·어린이' '어린이' '성인'
 '8명 (초등 5-6학년)' '도서관 이용자 누구나' '초등학생 및 어린이 가족'
 '초등 3~4학년 (2014~2015년생, 12명)' '초등학생 어린이 및 가족' '자녀가 있는 부모'
 '전체 (일부 프로그램 제외)' '초등 1~3학년 어린이' '6세~13세, 15명' '초등 3~4학년' '일반인'
 '초등 3~6학년/ 12명' '유아(2018년생 ~ 2020년생), 초등 1 ~ 6학년 및 동반 가족' '어린이를 포함한 10가족'
 '6~8세/ 15명' '누구나 (20명)' '일반 25명' '청소년, 성인' '성인(지역 청년 및 직장인)' '일반 30명'
 '6세~13세' '성인 (15명)' '유아' '초등 1~2학년/ 12명' '3~7세 유아 누구나(2018~2022년생)'
 '초등 1~2학년' '누구나 (선착순 80명)' '초등 3~6학년' '초등학교 1~2학년' '성인 (12명)' '지역주민'
 '초등 1-6학년 해봄이(요일별 6명)' '성인 20명' '은평구 주민 누구나(선착순 마감)'
 '초등학생 (은평구공공도서관 정회원)' '성인 누구나' '80명'
 '서울야외도서관 사업에 관심 있는 전국 자치단체 및 민간단체 관계자, 대학 소속 직원 및 학생' '프로그램별 상이'
 '성인 및 청소년' '초등 1~2학년 어린이 12명' '자녀를 둔 부모 및 양육자' '6-7세 유아' '초등학교 1~3학년 10명'
 '유아(2018~2019년생), 초등 1~6학년 및 동반 가족' '초등학생 1~3학년 (2015~2017) 15명'
 '누구나 (30명)' '상세 내용 참고' '유아 및 아동(각 수업 별 인원 상이)' '성인 (20명)' '강좌별 상이'
 '초등 4학년 이상~성인(8명)' '인도철학에 관심있는 성인 12명' '초등학생 3 ~ 4학년 12명' '성인 30명'
 '청소년 이상 누구나' '독서 동아리 회원 및 지역주민' '영유아와 양

In [88]:
grouped = events_whole.groupby('lib_name').count()
grouped = grouped.sort_values('event_link',ascending=False)
grouped

,event_link,event_info,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies,start_date,end_date
lib_name,,,,,,,,,,,,
구립증산정보도서관,324,324,324,324,324,324,324,324,183,324,324,324
서강도서관,84,84,84,84,84,84,84,84,0,84,84,84
마포구립서강도서관,55,55,55,55,55,55,55,55,54,55,55,55
서울도서관,48,48,48,48,48,48,48,48,47,48,48,48
구로구립 궁동어린이도서관,38,38,38,38,38,38,38,38,10,38,38,38
...,...,...,...,...,...,...,...,...,...,...,...,...
원당한옥마을도서관 「고즈넉한 북살롱: 자녀를 위한 문해력」\n2023-03-11 ~ 2023-03-18\n원당마을한옥도서관,1,1,1,1,1,1,1,1,0,1,1,1
유현준 교수와 함께하는 “공간의 미래―도서관과 광장” 강연\n2022-04-23 ~ 2022-04-23,1,1,1,1,1,1,1,1,0,1,1,1
은평구 도서관형 메이커 창작 문화 확산 사업,1,1,1,1,1,1,1,1,1,1,1,1


In [89]:
grouped.head(30)

,event_link,event_info,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies,start_date,end_date
lib_name,,,,,,,,,,,,
구립증산정보도서관,324,324,324,324,324,324,324,324,183,324,324,324
서강도서관,84,84,84,84,84,84,84,84,0,84,84,84
마포구립서강도서관,55,55,55,55,55,55,55,55,54,55,55,55
서울도서관,48,48,48,48,48,48,48,48,47,48,48,48
구로구립 궁동어린이도서관,38,38,38,38,38,38,38,38,10,38,38,38
강남구립논현도서관,28,28,28,28,28,28,28,28,28,28,28,28
구로구립,24,24,24,24,24,24,24,24,0,24,24,24
서울애니메이션센터,20,20,20,20,20,20,20,20,18,20,20,20
휘경행복도서관,19,19,19,19,19,19,19,19,15,19,19,19


In [2]:
import pandas as pd

events_df = pd.read_csv('./output/event_info.csv')
events_df

,event_link,event_info,lib_name,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies,start_date,end_date
0,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 7월 문화가 있는 영화관 영화상영안내\n2024-07-27 ~...,구립증산정보도서관,0,2층 소강당,2024-07-27 ~ 2024-07-27,14:00~16:00,도서관 이용자,무료,02-307-6030,7월 문화가 있는 영화관 영화상영안내\n 7월27일 토요일 오후 2시 영화...,2024-07-27,2024-07-27
1,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 2024 움직이는 책방(이동형 서울형 책방) 4회 : 콕콕콕\n202...,서울도서관,1,서울도서관 1층 생각마루,2024-07-27 ~ 2024-07-27,17:00-19:00,누구나(어른+아이 1팀으로 구성),무료,070-5413-5661,움직이는 책방(이동형 서울형 책방)에서\n우리동네 서울형 책방의 다채로운 문화행...,2024-07-27,2024-07-27
2,https://culture.seoul.go.kr/culture/culture/cu...,[서울도서관] 서울 문화의 밤(7월) 행사 - 신병주 작가와의 만남\n2024-07...,서울도서관,2,서울도서관 1층 생각마루,2024-07-26 ~ 2024-07-26,19:00 ~ 20:30,서울도서관 회원,무료,02-2133-0248,서울도서관 7월 마지막주 「서울 문화의 밤」 행사로 《서울의 자서전》 신병주 작가와...,2024-07-26,2024-07-26
3,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관][7월/팟캐스트] 서강도서관 피셜 47회 [일주일에 세 번, 동...",마포구립서강도서관,3,온라인,2024-07-26 ~ 2024-08-20,09:00 ~ 18:00,누구나,무료,02-3141-7053,"서강도서관피셜 마흔 일곱 번째 시간\n정경아 작가의 <일주일에 세 번, 동네문화...",2024-07-26,2024-08-20
4,https://culture.seoul.go.kr/culture/culture/cu...,[서초구립내곡도서관] 내곡도서관에서 여름을 즐기세요!\n2024-07-26 ~ 20...,서초구립내곡도서관,4,서초구립내곡도서관,2024-07-26 ~ 2024-08-31,프로그램 별 상이 / 내곡도서관 매 주 월요일 휴관,유아·어린이,"원데이베이킹 재료비 15,000원",02-3461-3072,SUMMER BREAK WITH LIBRARY & 도서관에서 여름나기 \n ...,2024-07-26,2024-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
941,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관]_[유아 독서동아리 모집] 책&별(책읽는 미래의 별)\n2021...,강남구립청담도서관,941,ZOOM 실시간 화상 강의,2021-03-25 ~ 2022-02-24,16:00~16:40,"미취학 아동 6~7세, 10명",없음,540-7042,NaN,2021-03-25,2022-02-24
942,https://culture.seoul.go.kr/culture/culture/cu...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,국립중앙도서관 실감서재\n2021-03-23 ~ 2022-12-31\n국립중앙도서관...,942,국립중앙도서관 디지털도서관 지하 3층 실감서재,2021-03-23 ~ 2022-12-31,10:00 ~ 17:00 (회차당 50분),전 연령층 [유아 및 어린이(만 12세 이하)는 성인 보호자 동반 필수],무료,NaN,"실감나는 콘텐츠, 체험하는 도서관!\n『실감서재』는 첨단 기술이 적용된 새로운 형태...",2021-03-23,2022-12-31
943,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 책퐁(책속으로 퐁당)\n2021-0...,강남구립청담도서관,943,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,11:00~12:00,2021년 기준 초등학생 1~2학년,없음,540-7042,NaN,2021-03-14,2022-02-13
944,https://culture.seoul.go.kr/culture/culture/cu...,[강남구립청담도서관] _[초등 독서동아리 모집] 마따동(마음이 따뜻해지는 동화)\n...,강남구립청담도서관,944,ZOOM 실시간 화상 수업,2021-03-14 ~ 2022-02-13,매월 둘째 주 일요일 (16:00~17:00),2021년 기준 초등학생 3~4학년,없음,540-7042,NaN,2021-03-14,2022-02-13


In [4]:
mask = (events_df['start_date'] >= '2023-01-01') & (events_df['start_date'] <= '2023-12-31')
filtered_events = events_df[mask]
filtered_events

In [5]:
filtered_events

,event_link,event_info,lib_name,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies,start_date,end_date
325,https://culture.seoul.go.kr/culture/culture/cu...,[마포구립서강도서관] 12-1월 세대교류 그림책 독서모임\n2023-12-28 ~ ...,마포구립서강도서관,325,서강도서관 3층 Book C,2023-12-28 ~ 2024-01-18,매주 목요일 19:00~12:00/ 4회 - 매주 토요일 14:00~16:00/ 3회,"청년(만19세~34세) 6명, 중년(만48세~60세) 6명",무료,02-3141-7053,2023~2024년 세대교류 그림책 독서모임 \n\n\n\n1. 프로그램명: 20...,2023-12-28,2024-01-18
326,https://culture.seoul.go.kr/culture/culture/cu...,"[마포구립서강도서관] 12월 팟캐스트 피셜 41회 [설자은, 금성으로 돌아오다]\n...",마포구립서강도서관,326,팟빵(팟캐스트),2023-12-28 ~ 2024-01-31,언제나,누구나,무료,02-3141-7053,"12월 팟캐스트 피셜 41회 설자은, 금성으로 돌아오다 \n\n정세랑 작가의 <설...",2023-12-28,2024-01-31
327,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 어린이 직업탐색 네 꿈을 찾아봐! 12월 - 인테리어 디자이너...,구립증산정보도서관,327,3층 어린이열람실,2023-12-27 ~ 2023-12-27,2023년 12월 27일(수) 오후 3시 ~ 5시,어린이 이용자 누구나 (24명),무료,02-307-6030,12월 어린이 직업탐색 네 꿈을 찾아봐! 인테리어 디자이너 \n\n매월 마지막 주...,2023-12-27,2023-12-27
328,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 사서 선생님과 함께하는 [두근두근 크리스마스 준비!]\n202...,구립증산정보도서관,328,지하 1층 문화강좌실,2023-12-21 ~ 2023-12-21,2023.12.21.(목) 오후 4시~5시,4~5세 유아 10명 (부모 동반 가능),무료,02-307-6030,사서 선생님과 함께하는 두근두근 크리스마스 준비! \n\n코앞으로 다가온 크리스마...,2023-12-21,2023-12-21
329,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 민들레씨 그림책 놀이터 [12/20(수) 멍멍댄스]\n2023...,구립증산정보도서관,329,지하 1층 문화강좌실,2023-12-20 ~ 2023-12-20,16:00,6~7세 유아 10명,무료,02-307-6030,민들레씨 그림책 놀이터 멍멍댄스 \n\n우리 도서관에서 진행 된 ＜은평마을 ...,2023-12-20,2023-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
615,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 2023 1월 꼬마북클럽 '오늘 해님이 안나온다면'\n2023...,구립증산정보도서관,615,지하 문화강좌실,2023-01-09 ~ 2023-01-16,오후 4시,6~7세 유아 10명,무료,NaN,2023 1월 꼬마북클럽\n오늘 해님이 안 나온다면\n\n대상 _ 6~7세 유아 1...,2023-01-09,2023-01-16
616,https://culture.seoul.go.kr/culture/culture/cu...,[서울애니메이션센터] 도심 속 문화체험공간! 만화도서관 & 애니소풍\n2023-01...,서울애니메이션센터,616,"서울애니메이션센터 1,2층",2023-01-03 ~ 2023-01-31,"만화의집10:00-20:00/애니소풍10:00-18:00(매주 월요일, 기타 지정일...",어린이/청소년/성인,"만화의집(무료) / 유료(애니소풍) 성인 4,000원, 어린이 6,000원",02-3455-8341,1층 만화의집 - 다양한 만화책과 영상을 보유한 만화도서관으로 명동의 새로운 핫플레...,2023-01-03,2023-01-31
617,https://culture.seoul.go.kr/culture/culture/cu...,[구로구립 궁동어린이도서관] 2023 겨울방학 프로그램\n2023-01-02 ~ 2...,구로구립 궁동어린이도서관,617,궁동어린이도서관,2023-01-02 ~ 2023-02-23,10:00-18:00,도서관 이용자,무료,NaN,궁동어린이도서관 2023년 겨울방학 프로그램 전체 안내\n\n그림책과 함께하는 심리...,2023-01-02,2023-02-23
618,https://culture.seoul.go.kr/culture/culture/cu...,[구립증산정보도서관] 인스타그램 이벤트 ‘독서할계묘’\n2023-01-02 ~ 20...,구립증산정보도서관,618,도서관 인스타그램,2023-01-02 ~ 2023-01-14,1월 2일(월) ~ 1월 14일(토),도서관 이용자 누구나,무료,NaN,인스타그램 1월 이벤트 \n\n”독서할계묘”\n2023년을 맞이하여 진행하는 독서 ...,2023-01-02,2023-01-14


In [6]:
grouped = filtered_events.groupby('lib_name').count()
grouped = grouped.sort_values('event_link',ascending=False)
grouped

,event_link,event_info,events_num,events_place,events_period,events_time,events_target,events_price,events_q,events_dummies,start_date,end_date
lib_name,,,,,,,,,,,,
구립증산정보도서관,125,125,125,125,125,125,125,125,83,125,125,125
서울도서관,19,19,19,19,19,19,19,19,19,19,19,19
구로구립글마루한옥어린이도서관,11,11,11,11,11,11,11,11,8,11,11,11
마포구립서강도서관,9,9,9,9,9,9,9,9,9,9,9,9
도봉기적의도서관,9,9,9,9,9,9,9,9,3,9,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...
은평뉴타운도서관,1,1,1,1,1,1,1,1,1,1,1,1
장안어린이도서관,1,1,1,1,1,1,1,1,0,1,1,1
종로문화재단,1,1,1,1,1,1,1,1,0,1,1,1


---